<a href="https://colab.research.google.com/github/dgromann/ProgrammingForTranslators/blob/master/colab/4_Beispiel_Termextraktion_L%C3%B6sungen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmieren für ÜbersetzerInnen - Beispiel Termextraktion




Als erstes Beispiel nach der Einführung schreiben wir ein simples Programm zur Termextraktion unter Verwendung des statistischen Maßes TF-IDF. TF-IDF steht für Termhäufigkeit (TF) und Inverse Dokumentenhäufigkeit (IDF) und ist ein Maß zur Berechnung der Gewichtung eines Wortes/einer Phrase in einem Dokument -  damit können fachsprachliche Benennungen extrahiert werden.

Unser Beispielkorpus besteht aus den folgenden drei Sätzen: 


```
document1 = "Sendung von Gold in einem Container." 
document2= "Lieferung von Silber in einem silbernen LKW angekommen." 
document3 = "Sendung von Gold in einem LKW angekommen."
```

Anstatt diese Sätze direkt im Notebook zu schreiben, laden wir drei Dokumente in unser Programm. Speichern Sie dazu die drei Textdateien in dem Github-Repository im Ordner "tutorial" (document1.txt document2.txt und documen3.txt) lokal auf Ihrem Rechner ab und laden Sie die Datein hoch wenn die Schaltfläche "Browse" erscheint beim Ausführen der nachstehende Code-Zelle:

In [2]:
# Datein in Google Colab laden
# Alternativ können Sie die drei Zeilen oben mit dem Text auch direkt als 
# Variablen in den Code schreiben anstatt die drei Dokumente hochzuladen
from google.colab import files
files.upload()

corpus = []

document1 = open("document1.txt")
document2= open("document2.txt")
document3 = open("document3.txt")

documents = [document1, document2, document3]

# Die Funktion strip() dient dazu das Symbol \n für die Markierung des Zeilenendes  
# und alle zusätzlichen Leerzeichen vor oder nach der Zeile zu entfernen
for document in documents:
  for sentence in document.readlines():
    corpus.append(sentence.strip())

#Wie Sie sehen wird für jedes Dokument eine Liste erstellt, die jede Zeile (in diesem Fall nur eine), des Dokuments enthält.
print(corpus)

Saving document3.txt to document3.txt
Saving document2.txt to document2.txt
Saving document1.txt to document1.txt
['Sendung von Gold in einem Container.', 'Lieferung von Silber in einem silbernen LKW angekommen.', 'Sendung von Gold in einem LKW angekommen.']


Um spaCy auch in diesem Notebook verwenden zu können, müssen wir spaCy und auch das deutsche Sprachpaket wieder laden:

In [3]:
# spaCy in Colab/Binder installieren - für Anaconda siehe spacy Installatiosnanweisungen
!pip install -U spacy
!python -m spacy download en
!python -m spacy download de

import spacy
nlp = spacy.load('de')

     |████████████████████████████████| 10.4MB 4.6MB/s 
     |████████████████████████████████| 122kB 54.3MB/s 
     |████████████████████████████████| 2.2MB 39.4MB/s 
     |████████████████████████████████| 3.7MB 33.7MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 12.0MB 803kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011742 sha256=7346a3299eced4d48165aab937172d8eac52d31921d2a8a790216e0176522bc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-psl4s693/wheels/6a/47/fb/6b5a0b89

Wir verwenden hier die Vorverarbeitungsmethode aus dem letzten Beispiel. Da diese in einem anderen Notebook steht, müssen wir Sie hier noch einmal wiedergeben: 

In [0]:
def preprocessing(sentence):
  sentence = sentence.lower() 
  # Wir fügen auch gleich ein paar weitere POS-Tags für Konjunktionen, etc. hinzu 
  pos_to_be_removed =['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
  text_out = []
  # Tokenisiert und weitere Vorverarbeitung
  doc = nlp(sentence)
  for token in doc:
    # POS-Tags überprüft und nur jene die nicht in "pos_to_be_removed" zu finden sind berücksichtigen
    if token.pos_ not in pos_to_be_removed :
      #Lemmatisierung
      lemma = token.lemma_
      text_out.append(lemma)  
  return text_out

**Aufgabe:** <br> 
Rufen Sie die oben definierte Methode `preprocessing` für jeden Satz des Korpus `corpus` auf und speichern Sie das Ergebnis in eine neue Liste `preprocessed`:

In [6]:
# Neue Liste preprocessed 
preprocessed = []

# Schreiben Sie hier Ihre Code - bedenken Sie dass corpus eine Liste ist 
for sentence in corpus:
  prep_sentence = preprocessing(sentence)
  preprocessed.append(prep_sentence)

# Das Resultat ist eine eigene Liste an Worte für jedes Dokument
# Die nachstehende Zeile fügt die einzelnen Worte für jedes Dokument wieder zu Sätzen zusammen
preprocessed = [" ".join(x) for x in preprocessed] 
print(preprocessed)

['sendung gold container', 'lieferung silber silbern lkw ankommen', 'sendung gold lkw ankommen']


# TF-IDF
Anstelle der manuellen Berechnung der TF-IDF Werte, verwenden wir eine bereitgestellte Library für maschinelles Lernen names `sklearn`. 



In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed)
word_index = vectorizer.vocabulary_

print("Wortindex: ", word_index)
print("TF-IDF Matrix: ")
print("(Dokumentennummer, Termindex) TF-IDF Weight")
print(tfidf_matrix)

Wortindex:  {'sendung': 5, 'gold': 2, 'container': 1, 'lieferung': 3, 'silber': 6, 'silbern': 7, 'lkw': 4, 'ankommen': 0}
TF-IDF Matrix: 
(Dokumentennummer, Termindex) TF-IDF Weight
  (0, 1)	0.680918560398684
  (0, 2)	0.5178561161676974
  (0, 5)	0.5178561161676974
  (1, 0)	0.3730219858594306
  (1, 4)	0.3730219858594306
  (1, 7)	0.49047908420610337
  (1, 6)	0.49047908420610337
  (1, 3)	0.49047908420610337
  (2, 0)	0.5
  (2, 4)	0.5
  (2, 2)	0.5
  (2, 5)	0.5


Der nachstehende Code extrahiert die `n` wichtigsten Terme aus unserem Mini-Korpus. 



**Frage:** <br> 
Würden Sie sagen, dass diese Auswahl für diesen Mini-Korpus Sinn macht? Stellen diese Worte tatsächlich wichtige semantische Bestandteile unseres Korpus dar?

In [8]:
import numpy as np

feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
print(top_n)

['sendung' 'lkw' 'gold']


Um die wichtigsten extrahieren Termini in einer Datei speichern zu können gehen wir wie folgt vor: 

In [0]:
# Speichern Sie die resultierende Datei in Ihrem lokalen Dateiverzeichnis
with open('results.txt', 'w') as f:
    for item in top_n:
        f.write("%s\n" % item)

files.download("results.txt")